In [1]:
import requests
import geopandas as gpd
import pandas as pd
import osmnx as ox
import folium
from osm2geojson import json2geojson
from shapely.ops import unary_union

/Users/jasper/Documents/Python_projects/data science/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Get boundary polygon by name
gdf = ox.geocode_to_gdf("Städteregion Aachen, Germany")

# Save to GeoJSON
gdf.to_file("staedteregion_aachen.geojson", driver="GeoJSON")

# Belgium – Province of Liège
liege = ox.geocode_to_gdf("Liège, Belgium")
liege.to_file("province_liege.geojson", driver="GeoJSON")

# Belgium – Province of Limburg
limburg_be = ox.geocode_to_gdf("Limburg, Belgium")
limburg_be.to_file("province_limburg_be.geojson", driver="GeoJSON")

# Belgium – German-speaking Community
# This is trickier – it's not an official province, but we can use the general name
german_speaking_be = ox.geocode_to_gdf("Deutschsprachige Gemeinschaft, Belgium")
german_speaking_be.to_file("german_speaking_community_be.geojson", driver="GeoJSON")

# Netherlands – Southern Limburg (broad area up to Roermond)
# Instead of just "Limburg", we clip down to the southern part
# You can adjust if needed – this includes Maastricht, Heerlen, Parkstad, Roermond
limburg_nl = ox.geocode_to_gdf("Limburg, Netherlands")
limburg_nl.to_file("province_limburg_nl.geojson", driver="GeoJSON")
# District of Düren
dueren = ox.geocode_to_gdf("Kreis Düren, Germany")
dueren.to_file("dueren.geojson", driver="GeoJSON")

# District of Euskirchen
euskirchen = ox.geocode_to_gdf("Kreis Euskirchen, Germany")
euskirchen.to_file("euskirchen.geojson", driver="GeoJSON")

# District of Heinsberg
heinsberg = ox.geocode_to_gdf("Kreis Heinsberg, Germany")
heinsberg.to_file("heinsberg.geojson", driver="GeoJSON")


In [3]:
# List of individual GeoJSON files
files = [
    "staedteregion_aachen.geojson",
    "dueren.geojson",
    "euskirchen.geojson",
    "heinsberg.geojson",
    "province_liege.geojson",
    "province_limburg_be.geojson",
    "german_speaking_community_be.geojson",
    "province_limburg_nl.geojson"
]

# Load and concatenate all regions
gdfs = [gpd.read_file(file) for file in files]
combined = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)

# Dissolve into a single multipolygon
euregio = combined.dissolve()

# Optional: Reset index and name the region
euregio = euregio.reset_index(drop=True)
euregio["name"] = "Euregio Meuse-Rhine"

# Save as single GeoJSON
euregio.to_file("euregio_meuse_rhine.geojson", driver="GeoJSON")


In [5]:
df = pd.read_csv('pdh_data.csv')

/var/folders/81/vvxrgbd107g667p_2ds7g5nh0000gn/T/ipykernel_31499/1459665880.py:1: DtypeWarning: Columns (3,4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('pdh_data.csv')


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1071401 entries, 0 to 1071400
Data columns (total 21 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   category                1071401 non-null  object 
 1   lat                     1032153 non-null  float64
 2   lon                     1032153 non-null  float64
 3   name                    969616 non-null   object 
 4   city                    427142 non-null   object 
 5   country                 1069984 non-null  object 
 6   type                    1071401 non-null  object 
 7   sector                  6012 non-null     object 
 8   source_type             1071401 non-null  object 
 9   data_collection_method  1040759 non-null  object 
 10  source_text             1070842 non-null  object 
 11  source_url              1049446 non-null  object 
 12  dataset_id              1071401 non-null  int64  
 13  dataset_name            1071401 non-null  object 
 14  pf

In [17]:
df['source_type'].value_counts()

source_type
Authorities                 1064483
Scientific article             3957
OSINT                          2477
Company                         198
Press inquiry                   150
Journalism investigation        102
NGO                              34
Name: count, dtype: int64

In [18]:
df['type'].value_counts()

type
Sampling location                   1058740
Industrial site                        6197
Waste management site                  3899
Firefighting incident / training       1067
Airport                                 936
Military site                           542
PFAS production facility                 20
Name: count, dtype: int64

In [22]:
df['pfas_values'].value_counts().head(10)

pfas_values
[]                                                                                            12648
[{"cas_id": "54910-89-3", "unit": "ng/l", "substance": "fluoxetine", "less_than": 5.0}]        5953
[{"cas_id": "1763-23-1", "unit": "ng/l", "substance": "PFOS", "less_than": 100.0}]             4882
[{"cas_id": "142459-58-3", "unit": "ng/l", "substance": "Flufenacet", "less_than": 10.0}]      2837
[{"cas_id": "1763-23-1", "unit": "ng/l", "substance": "PFOS", "less_than": 40.0}]              2653
[{"cas_id": "1763-23-1", "unit": "ng/l", "substance": "PFOS", "less_than": 10.0}]              2439
[{"cas_id": "27314-13-2", "unit": "ng/l", "substance": "Norflurazon", "less_than": 40.0}]      2252
[{"cas_id": "27314-13-2", "unit": "ng/l", "substance": "Norflurazon", "less_than": 20.0}]      2241
[{"cas_id": "754-91-6", "unit": "ng/l", "substance": "FOSA", "less_than": 2.55}]               2100
[{"cas_id": "83164-33-4", "unit": "ng/l", "substance": "Diflufenican", "less_than": 20.0

In [23]:
df['pfas_sum'].value_counts().head(10)

pfas_sum
0.0     704273
2.0       9416
3.0       8543
10.0      7845
6.0       6940
5.0       6794
4.0       6673
1.0       6440
20.0      5643
7.0       5319
Name: count, dtype: int64

In [20]:
df['data_collection_method'].value_counts()

data_collection_method
Public data - API         673586
Public data - Download    179886
Public data - Scraping    166723
Press inquiry              16151
FOI                         2681
OSINT                       1732
Name: count, dtype: int64

In [29]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Load GeoJSON
geojson_gdf = gpd.read_file("euregio_meuse_rhine.geojson")

# Drop rows with missing lat/lon
df = df.dropna(subset=['lat', 'lon'])

# Convert to GeoDataFrame
points_gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['lon'], df['lat']),
    crs="EPSG:4326"
)

# Match CRS
if geojson_gdf.crs != points_gdf.crs:
    geojson_gdf = geojson_gdf.to_crs(points_gdf.crs)

# Spatial join: filter points within the GeoJSON area
joined = gpd.sjoin(points_gdf, geojson_gdf, predicate='within', how='inner')




In [36]:
# Drop NaN first for clean filtering
pfas_nonan = joined['pfas_sum'].dropna()

# Count how many > 10
count_gt_10 = (pfas_nonan > 10).sum()

# Count how many <= 10
count_le_10 = (pfas_nonan <= 10).sum()

print(f"Count of pfas_sum > 10: {count_gt_10}")
print(f"Count of pfas_sum <= 10: {count_le_10}")
print(count_gt_10/count_le_10)

Count of pfas_sum > 10: 2124
Count of pfas_sum <= 10: 17822
0.11917854337335877


In [37]:
# Drop NaN first for clean filtering
pfas_nonan = df['pfas_sum'].dropna()

# Count how many > 10
count_gt_10 = (pfas_nonan > 10).sum()

# Count how many <= 10
count_le_10 = (pfas_nonan <= 10).sum()

print(f"Count of pfas_sum > 10: {count_gt_10}")
print(f"Count of pfas_sum <= 10: {count_le_10}")
print(count_gt_10/count_le_10)


Count of pfas_sum > 10: 204562
Count of pfas_sum <= 10: 814943
0.251013874590002


In [38]:
import folium
import geopandas as gpd
import pandas as pd

# Assuming your df and geojson_gdf are loaded already, and points_gdf has geometry
points_gdf = joined
# Filter points with pfas_sum > 10 and drop missing geometries
points_high_pfas = points_gdf[(points_gdf['pfas_sum'] > 10) & (~points_gdf['geometry'].isna())]

# Calculate center of the GeoJSON area to center the map nicely
center = geojson_gdf.geometry.unary_union.centroid
map_center = [center.y, center.x]

# Create the Folium map centered on your GeoJSON area
m = folium.Map(location=map_center, zoom_start=9)

# Add GeoJSON borders
folium.GeoJson(
    geojson_gdf.geometry.__geo_interface__,
    name="Borders"
).add_to(m)

# Add markers for points where pfas_sum > 10
for _, row in points_high_pfas.iterrows():
    lat = row.geometry.y
    lon = row.geometry.x
    popup_text = f"PFAS Sum: {row['pfas_sum']}"
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        color='red',
        fill=True,
        fill_opacity=0.7,
        popup=popup_text
    ).add_to(m)

# Display the map
m


/var/folders/81/vvxrgbd107g667p_2ds7g5nh0000gn/T/ipykernel_31499/1968194163.py:11: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = geojson_gdf.geometry.unary_union.centroid
